In [1]:
from argparse import Namespace
import xml.etree.ElementTree as ET
import os
from json import dumps
import xmltodict

In [2]:
def xml_to_json(xml_filepath):
    xml_filepath = os.path.expandvars(xml_filepath)
    with open(xml_filepath) as f:
        s = f.read()
    return xmltodict.parse(s)

lyt = xml_to_json('$PDK_ROOT/$PDK/libs.tech/klayout/tech/$PDK.lyt')
lyt

{'technology': {'name': 'sky130',
  'description': 'SkyWater 130nm technology',
  'group': None,
  'dbu': '0.001',
  'base-path': None,
  'original-base-path': '$PDK_ROOT/$PDK/libs.tech/klayout',
  'layer-properties_file': 'sky130A.lyp',
  'add-other-layers': 'true',
  'default-grids': '0.01,0.005!',
  'reader-options': {'gds2': {'box-mode': '1',
    'allow-big-records': 'true',
    'allow-multi-xy-records': 'true'},
   'common': {'create-other-layers': 'true',
    'layer-map': 'layer_map()',
    'enable-properties': 'true',
    'enable-text-objects': 'true'},
   'lefdef': {'read-all-layers': 'true',
    'layer-map': 'layer_map()',
    'dbu': '0.001',
    'produce-net-names': 'true',
    'net-property-name': '#1',
    'produce-inst-names': 'true',
    'inst-property-name': '#1',
    'produce-pin-names': 'false',
    'pin-property-name': '#1',
    'produce-cell-outlines': 'true',
    'cell-outline-layer': 'OUTLINE',
    'produce-placement-blockages': 'true',
    'placement-blockage-laye

In [3]:
from sqlalchemy import create_engine, Table, Column, ForeignKey, Integer, Boolean, Float, String, Text
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy.sql.expression import text

Base = declarative_base()

class Technology(Base):
    __tablename__     = 'technologies'
    id                = Column(Integer, primary_key=True)
    
    name              = Column(String(255), nullable=False, unique=True)
    description       = Column(Text, nullable=True)
    dbu               = Column(Float, nullable=False)
    add_other_layers  = Column(Boolean, nullable=False)
    
    writer_options    = relationship('WriterOptions', uselist=False, back_populates='technology')
    connections       = relationship('Connection', back_populates='technology')
    symbols           = relationship('Symbol', back_populates='technology')
    

class WriterOptions(Base):
    __tablename__    = 'writer_options'
    id               = Column(Integer, primary_key=True)
    
    libname          = Column(String(255), nullable=False)
    max_vertex_count = Column(Integer, nullable=False)
    write_timestamps = Column(Boolean, nullable=False)
    
    technology_id    = Column(Integer, ForeignKey('technologies.id'))
    technology       = relationship('Technology', back_populates='writer_options')

class Connection(Base):
    __tablename__ = 'connections'
    id            = Column(Integer, primary_key=True)
    
    a_symbol_id   = Column(Integer, ForeignKey('symbols.id'))
    a_symbol      = relationship('Symbol', foreign_keys=[a_symbol_id], back_populates='as_a_symbol_connections')
    
    via_symbol_id = Column(Integer, ForeignKey('symbols.id'))
    via_symbol    = relationship('Symbol', foreign_keys=[via_symbol_id], back_populates='as_via_symbol_connections')
    
    b_symbol_id   = Column(Integer, ForeignKey('symbols.id'))
    b_symbol      = relationship('Symbol', foreign_keys=[b_symbol_id], back_populates='as_b_symbol_connections')
    
    technology_id = Column(Integer, ForeignKey('technologies.id'))
    technology    = relationship('Technology', back_populates='connections')

SymbolPhysicalLayerPairs = Table(
    'symbol_physical_layer_pairs', 
    Base.metadata,
    Column('symbol_id', Integer, ForeignKey('symbols.id'), primary_key=True),
    Column('physical_layer_id', Integer, ForeignKey('physical_layers.id'), primary_key=True),
)

class Symbol(Base):
    __tablename__             = 'symbols'
    id                        = Column(Integer, primary_key=True)
    
    name                      = Column(String(255), nullable=False)
    
    technology_id             = Column(Integer, ForeignKey('technologies.id'))
    technology                = relationship('Technology', back_populates='symbols')
    
    as_a_symbol_connections   = relationship('Connection', foreign_keys=[Connection.a_symbol_id], back_populates='a_symbol')
    as_via_symbol_connections = relationship('Connection', foreign_keys=[Connection.via_symbol_id], back_populates='via_symbol')
    as_b_symbol_connections   = relationship('Connection', foreign_keys=[Connection.b_symbol_id], back_populates='b_symbol')

    physical_layers           = relationship('PhysicalLayer', secondary=SymbolPhysicalLayerPairs, back_populates='symbols')
    
class PhysicalLayer(Base):
    __tablename__ = 'physical_layers'
    id            = Column(Integer, primary_key=True)

    layer         = Column(Integer, nullable=False)
    datatype      = Column(Integer, nullable=False)
    negative      = Column(Boolean, server_default=text('0'), nullable=False)
    
    symbols       = relationship('Symbol', secondary=SymbolPhysicalLayerPairs, back_populates='physical_layers')

engine = create_engine('sqlite:///pdk_knowledge_base.db', echo=True)
Base.metadata.create_all(engine)

2025-09-28 14:24:45,097 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-28 14:24:45,097 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("technologies")
2025-09-28 14:24:45,097 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-28 14:24:45,098 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("technologies")
2025-09-28 14:24:45,098 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-28 14:24:45,099 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("writer_options")
2025-09-28 14:24:45,099 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-28 14:24:45,099 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("writer_options")
2025-09-28 14:24:45,099 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-28 14:24:45,100 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("connections")
2025-09-28 14:24:45,100 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-28 14:24:45,100 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("connections")
2025-09-28 14:24:45,101 INFO sqlal